In [1]:
import requests
import pandas as pd
import numpy as np
from keys import OWM_key

city = "Busan"
country = "KR"
response = requests.get(f'http://api.openweathermap.org/data/2.5/forecast/?q={city},{country}&appid={OWM_key}&units=metric&lang=en')